In [11]:
import util as ut
import recomender.trivial as trivial
import evaluate as eval

import json
import numpy as np
from sklearn.model_selection import train_test_split as sk_train_test_split
from operator import itemgetter


In [12]:
#data = ut.get_data(ut.RATINGS)



def read_ratings(test_size=0.33):
    test_size += 0.01

    with open(ut.RATINGS, 'r') as json_file:
        json_list = list(json_file)

    # Test Dict
    test_user_dict = {}
    test_user_idx=0
    test_item_dict = {}
    test_item_idx=0
    test_ratings_dict = {}
    
    # Train Dict
    train_user_dict = {}
    train_user_idx=0
    train_item_dict = {}
    train_ratings_dict = {}
    train_item_idx=0

    counter = 1
    for json_str in json_list:
        result = json.loads(json_str)
        if (not isinstance(result, dict)):
            continue
        # print(counter, test_size, counter * test_size)
        item_id = result['ItemId']
        user_id = result['UserId']
        rating  = result['Rating']
        if counter * test_size > 1:
            counter = 0
            
            if item_id not in test_item_dict: 
                test_item_dict[item_id] = {'Idx':test_item_idx, 'Rating_sum':0, 'Users':[]}
                test_item_idx += 1
            test_item_dict[item_id]['Rating_sum'] += rating
            test_item_dict[item_id]['Users'].append(user_id)
            if user_id not in test_user_dict:
                test_user_dict[user_id] = {'Idx':test_user_idx, 'Rating_sum':0, 'Items':[]}
                test_user_idx += 1
            test_user_dict[user_id]['Rating_sum'] += rating
            test_user_dict[user_id]['Items'].append(item_id) 

            test_ratings_dict[user_id + ':' + item_id] = {'Timestamp':result['Timestamp'], 'Rating':rating}
        else:
            counter += 1
            if item_id not in train_item_dict: 
                train_item_dict[item_id] = {'Idx':train_item_idx, 'Rating_sum':0, 'Users':[]}
                train_item_idx += 1
            train_item_dict[item_id]['Rating_sum'] += rating
            train_item_dict[item_id]['Users'].append(user_id)
            if user_id not in train_user_dict:
                train_user_dict[user_id] = {'Idx':train_user_idx, 'Rating_sum':0, 'Items':[]}
                train_user_idx += 1
            train_user_dict[user_id]['Rating_sum'] += rating
            train_user_dict[user_id]['Items'].append(item_id) 
            train_ratings_dict[user_id + ':' + item_id] = {'Timestamp':result['Timestamp'], 'Rating':rating}

    return [test_user_dict, test_item_dict, test_ratings_dict], [train_user_dict, train_item_dict, train_ratings_dict]


test, train = read_ratings()
len(test[2]), len(train[2])

(164930, 494790)

In [13]:


def create_matrix(user_dict, item_dict,  ratings_dict):
    # # # Create dict for idx of user and item
    # # user_dict = {}
    # # item_dict = {}
    # # user_idx, item_idx = 0, 0
    # # for user_id, item_id, rating in data:
    # #     if user_id not in user_dict:
    # #         user_dict[user_id] = user_idx
    # #         user_idx += 1
    # #     if item_id not in item_dict:
    # #         item_dict[item_id] = item_idx
    # #         item_idx += 1

    # Create rating matrix
    rating_matrix = [[np.nan]*(len(item_dict))]*(len(user_dict)) 
    print(len(rating_matrix), len(user_dict), len(item_dict))
    for pair in ratings_dict.keys():
        user_id, item_id = pair.split(':')
        # print([user_dict[user_id],item_dict[item_id]])
        rating_matrix[user_dict[user_id]['Idx']][item_dict[item_id]['Idx']] = ratings_dict[pair]['Rating']

    return rating_matrix


train_user_dict, train_item_dict, train_ratings_dict = train
rating_matrix = create_matrix(train_user_dict, train_item_dict, train_ratings_dict)

45896 45896 26230


In [20]:
def test_to_recomendation(test):
  test_user_dict, test_item_dict, test_ratings_dict = test
  
  ratings = []
  for key in test_ratings_dict.keys():
    ratings.append([key, test_ratings_dict[key]['Rating']] )
    
  ratings = sorted(ratings, key=itemgetter(1), reverse=True)
  
  user_dict = {}
  for pair, rating in ratings:
    user, item = pair.split(':')
    if user not in user_dict: user_dict[user] = []
    user_dict[user].append(item)
  recomendations = []
  for user in user_dict.keys():
    [recomendations.append(user + ':' + item + '\n') for item in user_dict[user]]
  
  return recomendations
test_recomendation = test_to_recomendation(test)


In [21]:

# def trivial_prediction(train, targets):
#   # rating_keys = train[2].keys()
#   train_user_dict, train_item_dict, train_ratings_dict = train
  

#   mean_rating = 0
#   for key in train[2]:
#     mean_rating += train[2][key]['Rating']
#   mean_rating = mean_rating/len(train[2])



#   predictions = []
#   for pair in targets:
#     _tuser, _titem = pair.split(":")
#     if _titem in train_item_dict:
#       _item_dict = train_item_dict[_titem]
#       prediction = _item_dict['Rating_sum']/len(_item_dict['Users'])
#     else:
#       prediction = mean_rating
#     predictions.append([_tuser, _titem, prediction])
  
#   predictions = sorted(predictions, key=itemgetter(2), reverse=True)  
#   return predictions

predictions = trivial.trivial_prediction(train, test[2].keys())

In [22]:

# def trivial_recomendation(train, targets):
#   predictions = trivial.trivial_prediction(train, targets)
#   recomend_dict = {}
  
#   for user, item, rating in predictions:

#     if user not in recomend_dict: recomend_dict[user] = []
#     recomend_dict[user].append(item)

#   with open(ut.RECOMENDATION, 'w') as recomendation_file:
#     recomendations = []
#     for user in recomend_dict.keys():
#       [recomendations.append(user + ':' + item) for item in recomend_dict[user]]
  
#     recomendation_file.writelines('UserId:ItemId\n')
#     recomendation_file.writelines(recomendations)
#   return recomendations


recomendation =trivial.trivial_recomendation(train, test[2].keys())
  

In [47]:
# from sklearn.metrics import ndcg_score
# def get_relevance(recomendation, expected):
#   expected_dict = {}
#   recomendation_dict = {}

#   # Create dict with user item and relevance
#   user_relevance = {}
#   for pair in expected:
#     user, item = pair[:-1].split(':')
#     if user not in expected_dict: 
#       expected_dict[user] = []
#       user_relevance[user] = 0
#     user_relevance[user] += 1
#     expected_dict[user].append([item, user_relevance[user]])
    
#   # create a dict with the pairs and relevance in relation to the user
#   user_relevance = {}
#   for pair in recomendation:
#     pair = pair[:-1]
#     user, item = pair.split(':')
#     if user not in user_relevance: user_relevance[user] = 0
#     user_relevance[user] += 1
    
#     recomendation_dict[pair] = user_relevance[user]
    
#   # match the order of the true relevance to the predicted relevance
#   true_relevance = []
#   expected_relevance = []
#   for user in expected_dict.keys():
#     for item_relevance in expected_dict[user]:
#       item, relevance = item_relevance
#       true_relevance.append(relevance)
#       expected_relevance.append(recomendation_dict[user + ':' + item])      




#   return true_relevance, expected_relevance

# def discount_cumulative_gain(recomendation, expected):
#   true_relevance, expected_relevance = get_relevance(recomendation, expected)
  
#   return ndcg_score(np.array([true_relevance]), np.array([expected_relevance]))

discount_cumulative_gain(recomendation, test_recomendation)

0.972025629109557

In [24]:
predictions

[['41f1f19176', '911ce25765', 10.0],
 ['b056eb1587', '8886a4bcfd', 10.0],
 ['b056eb1587', '8698e6d654', 10.0],
 ['74bba22728', 'fd5d61b60c', 10.0],
 ['b9141aff14', '28620520c7', 10.0],
 ['16e6a3326d', '607e138df9', 10.0],
 ['2bd7f907b7', '3cde7667c9', 10.0],
 ['5938b4d054', '0a85930bce', 10.0],
 ['5938b4d054', '2c618774f8', 10.0],
 ['5938b4d054', '65e130d75b', 10.0],
 ['5938b4d054', 'beb022a896', 10.0],
 ['5938b4d054', '7b55a40c47', 10.0],
 ['03255088ed', 'fd5d61b60c', 10.0],
 ['d756d3d2b9', 'fd5d61b60c', 10.0],
 ['a3d06db1f8', 'e04e402ad3', 10.0],
 ['5812f92450', '3cde7667c9', 10.0],
 ['334467d41d', 'fae7db1ee5', 10.0],
 ['0655f11744', '889f2e1442', 10.0],
 ['30192e936b', 'fd5d61b60c', 10.0],
 ['9bdb8b1faf', '90a9beec23', 10.0],
 ['9bdb8b1faf', 'ca79995ff5', 10.0],
 ['9bdb8b1faf', '9179fea862', 10.0],
 ['9bdb8b1faf', '21ad2a215f', 10.0],
 ['5e4450dc93', '5059784a61', 10.0],
 ['c79ec57a8e', 'fd5d61b60c', 10.0],
 ['a8ef1979ae', 'd3870236e4', 10.0],
 ['ab6439fa2d', 'b939860944', 10.0],
 

In [7]:
with open(ut.CONTENT, 'r') as json_file:
    json_list = list(json_file)


content_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    if (isinstance(result, dict)):
        content_dict[result['ItemId']] = result   

In [8]:
data

NameError: name 'data' is not defined